In [4]:
# 7.3.1 - LLMChain을 사용한 텍스트 생성
# 체인을 효과적으로 활용하기 위한 다양한 메서드가 제공되며, 각 메서드는 고유한 출력 형식을 제공

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import pprint

print("=" * 70)
print("7.3.1 LLMChain을 사용한 텍스트 생성")
print("=" * 70)

print("""
원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. llm_chain.apply() 메서드가 deprecated됨  
3. llm_chain.generate() 메서드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- apply() → batch() 메서드 사용
- generate() → batch() + LLMResult 구조 변경

핵심 개념:
- __call__: 체인을 사용하는 가장 간단한 방법
- apply(): 여러 입력을 동시에 전달하고 각 입력에 대해 리스트 형태의 결과 반환
- generate(): LLMResult 인스턴스를 반환해 줄 더 상세한 응답 제공
""")

# =============================================================================
# 프롬프트 템플릿 및 모델 설정
# =============================================================================
prompt_template = "What is a word to replace the following: {word}?"
prompt = PromptTemplate.from_template(prompt_template)

# Chat 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("프롬프트 템플릿:", prompt_template)
print("(다음 단어를 대체할 단어는 무엇인가?)")

# =============================================================================
# 최신 방식: RunnableSequence 사용
# =============================================================================
print("\n" + "=" * 60)
print("방법 1: 기본 invoke() 메서드 - 단일 입력 처리")
print("=" * 60)

# 최신 방식: prompt | llm
chain = prompt | llm

# 단일 입력 테스트
try:
    single_response = chain.invoke({"word": "artificial"})
    print(f"입력: artificial")
    print(f"출력: {single_response.content}")
    
except Exception as e:
    print(f"오류 발생: {e}")

# =============================================================================
# 배치 처리 (원서의 apply() 메서드 대체)
# =============================================================================
print("\n" + "=" * 60)
print("방법 2: batch() 메서드 - 여러 입력 동시 처리 (apply 대체)")
print("=" * 60)

# 입력 리스트
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

print("입력 데이터:")
for i, inp in enumerate(input_list, 1):
    print(f"{i}. {inp['word']}")

try:
    # 배치 처리 실행 (원서의 apply() 대체)
    batch_responses = chain.batch(input_list)
    
    print("\n배치 처리 결과:")
    for inp, response in zip(input_list, batch_responses):
        print(f"입력: {inp['word']} → 출력: {response.content.strip()}")
        
except Exception as e:
    print(f"배치 처리 오류: {e}")

# =============================================================================
# 기존 방식과 비교 (참고용)
# =============================================================================
print("\n" + "=" * 60)
print("기존 방식 vs 새로운 방식 비교")
print("=" * 60)

print("""
원서 코드 (Deprecated):
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)
responses = llm_chain.apply(input_list)          # ⚠️ Deprecated
result = llm_chain.generate(input_list)          # ⚠️ Deprecated

새로운 코드 (현재 권장):
chain = prompt | llm
responses = chain.batch(input_list)              # ✅ 권장
# generate() 기능은 batch()와 response 구조로 대체됨
""")

# =============================================================================
# 스트리밍 처리 (추가 기능)
# =============================================================================
print("\n" + "=" * 60)
print("방법 3: 스트리밍 처리 - 실시간 응답")
print("=" * 60)

print("스트리밍 예제 (단어: 'technology'):")

try:
    # 스트리밍으로 응답 받기
    print("실시간 응답: ", end="", flush=True)
    for chunk in chain.stream({"word": "technology"}):
        print(chunk.content, end="", flush=True)
    print("\n")
    
except Exception as e:
    print(f"스트리밍 오류: {e}")

# =============================================================================
# 응답 메타데이터 확인 (generate 기능 대체)
# =============================================================================
print("\n" + "=" * 60)
print("방법 4: 응답 메타데이터 확인 (generate 기능 대체)")
print("=" * 60)

try:
    # 배치 처리로 상세 정보 확인
    detailed_responses = chain.batch(input_list, return_exceptions=True)
    
    print("상세 응답 정보:")
    for i, (inp, response) in enumerate(zip(input_list, detailed_responses), 1):
        print(f"\n{i}. 입력: {inp['word']}")
        print(f"   응답 내용: {response.content}")
        print(f"   응답 타입: {type(response).__name__}")
        
        # 추가 메타데이터가 있다면 출력
        if hasattr(response, 'response_metadata'):
            print(f"   메타데이터: {response.response_metadata}")
            
except Exception as e:
    print(f"메타데이터 확인 오류: {e}")

# =============================================================================
# 다양한 프롬프트 템플릿 활용 예제
# =============================================================================
print("\n" + "=" * 60)
print("실용적인 활용 예제: 다양한 프롬프트 템플릿")
print("=" * 60)

# 번역 체인
translation_prompt = PromptTemplate.from_template(
    "Translate the following English word to Korean: {word}"
)
translation_chain = translation_prompt | llm

# 정의 체인
definition_prompt = PromptTemplate.from_template(
    "Provide a simple definition of: {word}"
)
definition_chain = definition_prompt | llm

# 동의어 체인 (원서 예제)
synonym_prompt = PromptTemplate.from_template(
    "What is a word to replace the following: {word}?"
)
synonym_chain = synonym_prompt | llm

test_words = ["artificial", "intelligence", "robot"]

try:
    print("다양한 체인 활용 결과:")
    for word in test_words:
        input_data = {"word": word}
        
        # 각 체인별로 처리
        synonym = synonym_chain.invoke(input_data)
        translation = translation_chain.invoke(input_data)
        definition = definition_chain.invoke(input_data)
        
        print(f"\n단어: {word}")
        print(f"동의어: {synonym.content.strip()}")
        print(f"한국어: {translation.content.strip()}")
        print(f"정의: {definition.content.strip()}")
        print("-" * 40)
        
except Exception as e:
    print(f"다양한 체인 활용 오류: {e}")

# =============================================================================
# 성능 및 활용 팁
# =============================================================================
print("\n" + "=" * 60)
print("성능 및 활용 팁")
print("=" * 60)

print("""
LLMChain 대체 방법별 특징:

1. invoke() - 단일 입력:
   - 가장 기본적인 방법
   - 한 번에 하나의 입력만 처리
   - 간단한 테스트나 단일 요청에 적합

2. batch() - 여러 입력:
   - 원서의 apply() 메서드 대체
   - 여러 입력을 동시에 효율적으로 처리
   - API 호출 최적화로 비용 절감

3. stream() - 실시간 응답:
   - 긴 응답을 실시간으로 받을 때 유용
   - 사용자 경험 개선
   - 챗봇이나 대화형 애플리케이션에 적합

4. 메타데이터 활용:
   - 원서의 generate() 기능 일부 대체
   - 응답의 추가 정보 확인 가능
   - 디버깅이나 로깅에 유용

성능 최적화 팁:
- 배치 처리로 API 호출 횟수 최소화
- temperature 값 조정으로 일관성 제어
- 적절한 프롬프트 길이 유지
- 오류 처리 및 재시도 로직 구현
""")

print("\n7.3.1 LLMChain 텍스트 생성 예제 완료!")

기존 환경 변수의 API 키를 사용합니다.
7.3.1 LLMChain을 사용한 텍스트 생성

원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. llm_chain.apply() 메서드가 deprecated됨  
3. llm_chain.generate() 메서드가 deprecated됨

수정 내용:
- LLMChain → RunnableSequence (prompt | llm)
- apply() → batch() 메서드 사용
- generate() → batch() + LLMResult 구조 변경

핵심 개념:
- __call__: 체인을 사용하는 가장 간단한 방법
- apply(): 여러 입력을 동시에 전달하고 각 입력에 대해 리스트 형태의 결과 반환
- generate(): LLMResult 인스턴스를 반환해 줄 더 상세한 응답 제공

프롬프트 템플릿: What is a word to replace the following: {word}?
(다음 단어를 대체할 단어는 무엇인가?)

방법 1: 기본 invoke() 메서드 - 단일 입력 처리
입력: artificial
출력: synthetic

방법 2: batch() 메서드 - 여러 입력 동시 처리 (apply 대체)
입력 데이터:
1. artificial
2. intelligence
3. robot

배치 처리 결과:
입력: artificial → 출력: synthetic
입력: intelligence → 출력: Cleverness
입력: robot → 출력: android

기존 방식 vs 새로운 방식 비교

원서 코드 (Deprecated):
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)
responses = llm_chain.apply(input_list)          # ⚠️ Deprecated
result = llm_chain.generate(input_list)  